# Steganalysis on Videos

### The following file contains functions for utilizing LSB encoding in order to hide a video into another video.
The two main functionalities are:
* encode_video - This function accepts two video files (mp4) as input. It splits each video into speparate frames and then applies the encoding procedure as defined in single_frame_steganalysis.py
* decode_video - This function accepts as input a video (mp4). It breaks down the video into a set of frames and performs the decoding process as defined in single_frame_steganalysis.py

In [ ]:
import numpy as np
import scipy as sc
from scipy import misc
import matplotlib.pyplot as plt
import os
import cv2
from single_frame_steganalysis import *
import scipy.misc

In [ ]:
def split_video_into_frames(video_path, path_out, vid_name):
    count = 0
    vidcap = cv2.VideoCapture(video_path)
    success,image = vidcap.read()
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000)) # added this line 
        success,image = vidcap.read()
        if success:
            cv2.imwrite( path_out + vid_name + "%d.jpg" % count, image) # save frame as JPEG file
            count = count + 1
    return count

In [ ]:
# SOURCE: https://stackoverflow.com/questions/44947505/how-to-make-a-movie-out-of-images-in-python
def convert_frames_to_video(images, image_folder, video_name='./data/encoded.mp4'):
#     images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, -1, 1, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

In [ ]:
def encode_video(secret_video_path, outer_video_path, path_out, secret_video_name, outer_video_name):
    if not(os.path.exists(path_out)):
    os.mkdir(path_out)
    num_secret_frames = split_video_into_frames(secret_video_path, path_out, secret_video_name)
    _ = split_video_into_frames(outer_video_path, path_out, outer_video_name)

    # iterate over each pair of frames
    for i in range(num_secret_frames):
        print(i)
        save_path = path_out + 'encoded_' + str(i) + '.jpg'
        visible_img_path = path_out + 'outer_' + str(i) + '.jpg'
        secret_img_path = path_out + 'secret_' + str(i) + '.jpg'
        encoded_frame = encode(visible_img_path, secret_img_path, save_path)
        scipy.misc.imsave(save_path, encoded_frame)
    encoded_frames = [img for img in os.listdir(path_out) if "encoded" in img] 
    encoded_frames = sorted(encoded_frames)
    convert_frames_to_video(encoded_frames, path_out)

In [ ]:
def decode_video(encoded_video, path_out, encoded_video_name):
    num_secret_frames = split_video_into_frames(encoded_video, path_out, encoded_video_name)
    for i in range(num_secret_frames):
        encoded_frame = path_out + encoded_video_name + str(i) + '.jpg'
        save_path = path_out + 'decoded_' + str(i) + '.jpg'
        decoded_frame = decode(encoded_frame, save_path)
        scipy.misc.imsave(save_path, decoded_frame)
    # convert frames back to video
    dencoded_frames = [img for img in os.listdir(path_out) if "dencoded" in img] 
    dencoded_frames = sorted(encoded_frames)
    convert_frames_to_video(dencoded_frames, path_out)

In [ ]:
secret_video_path = './data/secret.mp4'
outer_video_path = './data/outer.mp4'
path_out = './data/frames/'
secret_video_name = 'secret_'
outer_video_name = 'outer_'
encode_video(secret_video_path, outer_video_path, path_out, secret_video_name, outer_video_name)

In [ ]:
# decode video to extract secret video
encoded_video = './data/encoded.mp4'
path_out = './data/frames/'
encoded_video_name = 'encoded_'
decode_video(encoded_video, path_out, encoded_video_name)